# Preliminaries

In [84]:
import tensorflow as tf
import numpy as np

# Neural Net Definition

In [85]:
class my_neural_net(tf.keras.Model):
    def __init__(self):
        super(my_neural_net, self).__init__() # first of all, call the init method of the class it extends from
        self.all_in_one = tf.keras.Sequential([
                                               tf.keras.layers.Dense(20, activation='relu', input_shape=(1000,)),
                                               tf.keras.layers.Dense(5, activation='softmax')
        ])
    def call(self, x: tf.Tensor) -> tf.Tensor:
        output = self.all_in_one(x)
        return output

In [86]:
model = my_neural_net()

In [87]:
model.build((1,1000))

In [88]:
model.compile()

In [89]:
model(np.random.rand(2,1000))

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.05868606, 0.15921192, 0.07081547, 0.67329156, 0.03799501],
       [0.20250817, 0.11129512, 0.07229699, 0.49592248, 0.1179772 ]],
      dtype=float32)>

# Saving the Model

In [90]:
model.save('/model')

INFO:tensorflow:Assets written to: /model/assets


INFO:tensorflow:Assets written to: /model/assets


# [TFLITE](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter)

## Convert the Model to the ".tflite" File

Note: This new model can be later used on Android devices

In [91]:
converter = tf.lite.TFLiteConverter.from_saved_model('/model')

In [92]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [93]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]

In [94]:
tflite_model = converter.convert()

In [95]:
with open('/content/model.tflite', 'wb') as f:
  f.write(tflite_model)

## Load the Model and Use it (in Python)

In [96]:
interpreter = tf.lite.Interpreter('/content/model.tflite')

In [97]:
interpreter.allocate_tensors() # TensorFlow Lite pre-plans tensor allocations to optimize inference

In [98]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [99]:
type(input_details)

list

In [100]:
type(input_details[0])

dict

In [101]:
input_details[0].keys()

dict_keys(['name', 'index', 'shape', 'shape_signature', 'dtype', 'quantization', 'quantization_parameters', 'sparsity_parameters'])

In [102]:
input_details[0]['name']

'serving_default_input_1:0'

In [103]:
input_details[0]['shape_signature']

array([  -1, 1000], dtype=int32)

In [104]:
input_details[0]['quantization_parameters']

{'quantized_dimension': 0,
 'scales': array([], dtype=float32),
 'zero_points': array([], dtype=int32)}

In [105]:
input_details[0]['sparsity_parameters']

{}

In [106]:
input_details[0]['index']

0

In [107]:
input_shape = input_details[0]['shape'] # since the model has only one input
input_shape

array([   1, 1000], dtype=int32)

In [108]:
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [109]:
interpreter.invoke()

In [110]:
output_data = interpreter.get_tensor(output_details[0]['index'])

In [111]:
output_data

array([[0.12089428, 0.12201023, 0.08044706, 0.6186444 , 0.05800402]],
      dtype=float32)